In [ ]:
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from time import time

In [ ]:
TRAIN_TEXT_FILE_PATH = 'pushkin_stihi2.txt'

with open(TRAIN_TEXT_FILE_PATH) as text_file:
    text_sample = text_file.readlines()
text_sample = ' '.join(text_sample)

def text_to_seq(text_sample):
    char_counts = Counter(text_sample)
    char_counts = sorted(char_counts.items(), key = lambda x: x[1], reverse=True)

    sorted_chars = [char for char, _ in char_counts]
    print(sorted_chars)
    char_to_idx = {char: index for index, char in enumerate(sorted_chars)}
    idx_to_char = {v: k for k, v in char_to_idx.items()}
    sequence = np.array([char_to_idx[char] for char in text_sample])

    return sequence, char_to_idx, idx_to_char

sequence, char_to_idx, idx_to_char = text_to_seq(text_sample)

[' ', 'о', 'е', 'а', 'н', 'т', 'и', '\n', 'л', 'с', 'р', 'в', 'д', 'м', ',', 'у', 'к', 'ь', 'п', 'ы', 'й', 'я', 'г', 'б', 'з', '.', 'ч', 'ж', 'х', 'ш', 'ю', 'И', 'ц', 'В', 'Н', ';', 'П', 'О', 'С', '-', 'К', 'Т', ':', '"', '!', 'Д', '?', 'щ', 'М', 'Б', 'Ч', 'З', 'А', 'Е', 'Г', 'У', 'Л', 'э', 'Я', 'e', 'Р', 'ф', 'Х', 't', 'a', 'r', 'i', 'l', 'ъ', 'n', 'Ж', 'o', 'Ц', 's', 'u', 'Ш', 'c', 'd', 'Ф', 'm', 'Э', 'v', 'f', 'b', "'", 'g', 'p', 'N', 'M', 'h', 'S', 'T', 'A', 'Ю', 'k', 'Й', 'P', 'q', 'y', 'B', 'L', 'z', 'w', 'C', 'H', 'O', 'Y', 'E', 'Q', 'W', 'R', 'G', 'F', 'D', 'I']


In [ ]:
SEQ_LEN = 256
BATCH_SIZE = 16

def get_batch(sequence):
    trains = []
    targets = []
    for _ in range(BATCH_SIZE):
        batch_start = np.random.randint(0, len(sequence) - SEQ_LEN)
        chunk = sequence[batch_start: batch_start + SEQ_LEN]
        train = torch.LongTensor(chunk[:-1]).view(-1, 1)
        target = torch.LongTensor(chunk[1:]).view(-1, 1)
        trains.append(train)
        targets.append(target)
    return torch.stack(trains, dim=0), torch.stack(targets, dim=0)

In [ ]:
def evaluate(model, char_to_idx, idx_to_char, start_text=' ', prediction_len=200, temp=0.3):
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text

    _, hidden = model(train, hidden)

    inp = train[-1].view(-1, 1, 1)

    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char

    return predicted_text

In [ ]:
class TextRNN(nn.Module):

    def __init__(self, input_size, hidden_size, embedding_size, n_layers=1):
        super(TextRNN, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.input_size)

    def forward(self, x, hidden):
        x = self.encoder(x).squeeze(2)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)

    def backward(self, y, hidden):
        y = self.encoder(y).squeeze(2)
        out, (ht1, ct1) = self.lstm(y, hidden)
        out = self.dropout(out)
        y = self.fc(out)
        return y, (ht1, ct1)

    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = TextRNN(input_size=len(idx_to_char), hidden_size=128, embedding_size=128, n_layers=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    patience=5,
    verbose=True,
    factor=0.5
)

n_epochs = 1000
loss_avg = []
t1 = time()
for epoch in range(n_epochs):
    model.train()
    train, target = get_batch(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()

print("Training time:", time() - t1)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Loss: 2.9771893787384034
Loss: 2.421382436752319
Loss: 2.2737652587890627
Loss: 2.1830715131759644
Loss: 2.1125550174713137
Loss: 2.061673851013184
Loss: 2.005985336303711
Loss: 1.960284538269043
Loss: 1.92588458776474
Loss: 1.8997974801063537
Loss: 1.8683221387863158
Loss: 1.8568591928482057
Loss: 1.8241076016426085
Loss: 1.8170871829986572
Loss: 1.7936054348945618
Loss: 1.7830493116378785
Loss: 1.764551181793213
Loss: 1.744705855846405
Loss: 1.7455802536010743
Loss: 1.7327102851867675
Training time: 270.7499358654022


In [ ]:
model.eval()
print(evaluate(
    model,
    char_to_idx,
    idx_to_char,
    temp=0.3,
    prediction_len=200,
    start_text='Я помню чудное '
    )
)

Я помню чудное лет.
 Но не только в сад на свой она
 И все в так веселый постой,
 И в сердце все волна бы стало,
 И все на под ней как на своей
 В темно в нем верно в своей свой столов,
 И поверет не своей отвершины
